In [1]:
!pip install minsearch


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "Data Engineering Zoomcamp FAQ\nData Engineering Zoomcamp FAQ\nThe purpose of this document is to capture Frequently asked technical questions\nEditing guidelines:\nWhen adding a new FAQ entry, make sure the question is “Heading 2”\nFeel free to improve if you see something is off\nDon’t change the formatting in the Data document or add any visual “improvements” (make a copy for yourself first if you need to do it for whatever reason)\nDon’t change the pages format (it should be “pageless”)\nAdd name and date for reference, if possible\nThe next cohort starts January 13th 2025. More info at DTC.\nRegister before the course starts using this link.\nJoint the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When does the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [43]:
q = 'Can I get a certificate if I start very late?'

In [44]:
index.fit(documents)

In [45]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

results

[{'text': 'No, as long as you do the peer-reviewed capstone projects in time then you can get the certificate. You do not need to do the homeworks if you join late for example.',
  'section': 'Course Management Platform for Homeworks, Project and Certificate',
  'question': 'Certificate - Do I need to do the homeworks to get the certificate?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
  'section': 'Course Management Platform for Homeworks, Project and Certificate',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'No, late submissions are not allowed. But if the form is still not closed and it’s 

In [22]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

In [41]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()

In [42]:
response = client.chat.completions.create(
    model='llama-3.3-70b-versatile',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, you can get a certificate if you start very late, as long as you do the peer-reviewed capstone projects in time. You do not need to do the homeworks if you join late. However, please note that you need to be part of a "live" cohort to be eligible for a certificate, as certificates are not awarded for the self-paced mode.'

## Modularized Code

In [47]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [48]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [52]:
def llm(prompt):
    response = client.chat.completions.create(
        model='llama-3.3-70b-versatile',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [53]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [54]:
rag(query)

'To run Kafka, you need to create a virtual environment and install the required packages. \n\n1. Create a virtual environment by running: \n   `python -m venv env`\n\n2. Activate the virtual environment:\n   - On MacOS and Linux: `source env/bin/activate`\n   - On Windows: `env/Scripts/activate`\n\n3. Install the packages: \n   `pip install -r ../requirements.txt`\n\nAlternatively, if you\'re using Java Kafka, you can run the producer/consumer/kstreams/etc in the terminal by running:\n`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java` \n\nMake sure Docker images are up and running before creating the virtual environment. \n\nAlso, ensure you have the necessary dependencies installed. If you encounter a "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'", consider using `kafka-python-ng` instead by running `pip install kafka-python-ng`. \n\nIf you\'re experiencing permission issues, run `chmod +x build.sh` to grant execution 

In [55]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even after it has started. According to the FAQ, "Yes, even if you don\'t register, you\'re still eligible to submit the homework." However, be aware that there will be deadlines for turning in homework and the final project, so it\'s not recommended to leave everything for the last minute.'

In [58]:
documents[0]

{'text': "Data Engineering Zoomcamp FAQ\nData Engineering Zoomcamp FAQ\nThe purpose of this document is to capture Frequently asked technical questions\nEditing guidelines:\nWhen adding a new FAQ entry, make sure the question is “Heading 2”\nFeel free to improve if you see something is off\nDon’t change the formatting in the Data document or add any visual “improvements” (make a copy for yourself first if you need to do it for whatever reason)\nDon’t change the pages format (it should be “pageless”)\nAdd name and date for reference, if possible\nThe next cohort starts January 13th 2025. More info at DTC.\nRegister before the course starts using this link.\nJoint the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When does the course start?',
 'course': 'data-engineering-zoomcamp'}

## Elasticsearch

In [59]:
from elasticsearch import Elasticsearch

In [60]:
es_client = Elasticsearch('http://localhost:9200') 

In [61]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [62]:
documents[0]

{'text': "Data Engineering Zoomcamp FAQ\nData Engineering Zoomcamp FAQ\nThe purpose of this document is to capture Frequently asked technical questions\nEditing guidelines:\nWhen adding a new FAQ entry, make sure the question is “Heading 2”\nFeel free to improve if you see something is off\nDon’t change the formatting in the Data document or add any visual “improvements” (make a copy for yourself first if you need to do it for whatever reason)\nDon’t change the pages format (it should be “pageless”)\nAdd name and date for reference, if possible\nThe next cohort starts January 13th 2025. More info at DTC.\nRegister before the course starts using this link.\nJoint the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When does the course start?',
 'course': 'data-engineering-zoomcamp'}

In [63]:
from tqdm.auto import tqdm

In [64]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/1122 [00:00<?, ?it/s]

In [65]:
query = 'I just disovered the course. Can I still join it?'

In [68]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [69]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [70]:
rag(query)

"Yes, you can still join the course. Even if you don't register, you're still eligible to submit the homework. However, be aware that there will be deadlines for turning in homeworks and the final projects, so don't leave everything for the last minute."